In [1]:
import numpy as np
import pandas as pd
from random import shuffle
import cv2

In [2]:
train_data = np.load(r"data\train.npy", allow_pickle=True)

In [ ]:
for data in train_data:
    img = data[0]
    choice = data[1]
    cv2.imshow("test", img)
    print(choice)
    if cv2.waitKey(25) & 0XFF == ord('q'):
        cv2.destroyAllWindows()
        break

![title](data\test.PNG)

# Rebalancing data so that it's not all just driving forward examples

In [3]:
df = pd.DataFrame(train_data)
df.head()

,0,1
0,"[[28, 67, 97, 97, 97, 113, 116, 97, 97, 97, 97...","[0, 0, 0, 0, 1, 0, 0, 0, 0]"
1,"[[28, 67, 97, 97, 97, 113, 116, 97, 97, 97, 97...","[0, 0, 0, 0, 0, 0, 0, 0, 1]"
2,"[[28, 67, 97, 97, 97, 113, 116, 97, 97, 97, 97...","[0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,"[[28, 67, 97, 97, 97, 113, 116, 97, 97, 97, 97...","[0, 0, 0, 0, 0, 0, 0, 0, 1]"
4,"[[28, 67, 97, 97, 97, 113, 116, 97, 97, 97, 97...","[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [4]:
def output_to_key(vector):
    mapping = {
        0: 'w',
        1: 's',
        2: 'a',
        3: 'd',
        4: 'wa',
        5: 'wd',
        6: 'sa',
        7: 'sd',
        8: 'nk'
    }
    return mapping[np.argmax(vector)]

In [5]:
for value, count in df[1].value_counts().items():
    print(output_to_key(value), count)

w 6780
nk 5494
wd 1034
wa 1009
d 713
a 668
s 245
sa 46
sd 11


In [6]:
train_data2 = np.load(r"data\train2.npy", allow_pickle=True)

In [7]:
combined_df = pd.DataFrame(np.concatenate((train_data, train_data2), axis=0))

In [21]:
for value, count in combined_df[1].value_counts().items():
    print(output_to_key(value), count)

w 7047
nk 6113
wa 1335
wd 1139
d 867
a 790
s 737
sd 690
sa 282


In [22]:
from random import shuffle

train_data = np.concatenate((train_data, train_data2), axis=0)
lefts = []
rights = []
forwards = []
backwards = []
leftforwards = []
rightforwards = []
leftbacks = []
rightbacks = []
nokeys = []

w = [1,0,0,0,0,0,0,0,0]
s = [0,1,0,0,0,0,0,0,0]
a = [0,0,1,0,0,0,0,0,0]
d = [0,0,0,1,0,0,0,0,0]
wa =[0,0,0,0,1,0,0,0,0]
wd =[0,0,0,0,0,1,0,0,0]
sa =[0,0,0,0,0,0,1,0,0]
sd =[0,0,0,0,0,0,0,1,0]
nk =[0,0,0,0,0,0,0,0,1]

shuffle(train_data)

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == w:
        forwards.append([img, choice])
    elif choice == s:
        backwards.append([img, choice])
    elif choice == a:
        lefts.append([img, choice])
    elif choice == d:
        rights.append([img, choice])
    elif choice == wa:
        leftforwards.append([img, choice])
    elif choice == wd:
        rightforwards.append([img, choice])
    elif choice == sa:
        leftbacks.append([img, choice])
    elif choice == sd:
        rightbacks.append([img, choice])
    elif choice == nk:
        nokeys.append([img, choice])

In [24]:
import random

# List of all categories
categories = [lefts, rights, forwards, backwards, leftforwards, rightforwards, leftbacks, rightbacks, nokeys]

# Function to reduce the size of a list to 1000 elements
def reduce_to_1000(lst):
    while len(lst) > 1000:
        # Randomly remove an element
        index_to_remove = random.randint(0, len(lst) - 1)
        lst.pop(index_to_remove)

# Apply the function to each category
for category in categories:
    reduce_to_1000(category)

In [61]:
final_data = lefts + rights + forwards + backwards + leftforwards + rightforwards + leftbacks + rightbacks + nokeys
shuffle(final_data)

In [75]:
np.save('train_balanced.npy', np.array(final_data, dtype=object))

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initial categories
categories = [lefts, rights, backwards, leftforwards, rightforwards, leftbacks, rightbacks, nokeys]


# Define data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

# Assuming all images are of the same size, for example, 480x640
image_shape = (480, 640, 1)  # Change this based on your actual image size

for i, category in enumerate(categories):
    print(category)
    reshaped_images = np.array([img.reshape(image_shape) for img, _ in category])
    labels = np.array([label for _, label in category])

    augmented_count = 0
    for X_batch, _ in datagen.flow(reshaped_images, labels, batch_size=16):
        for augmented_image in X_batch:
            if augmented_count + len(category) < 7000:
                category.append((augmented_image, labels[0]))  # Append augmented image with original label
                augmented_count += 1
            else:
                break
        if augmented_count + len(category) >= 7000:
            break
    
    categories[i] = category  # Update the original category with augmented data

MemoryError: Unable to allocate 37.5 MiB for an array with shape (32, 480, 640, 1) and data type float32

In [ ]:

forwards = forwards[:len(lefts)][:len(rights)]
lefts = lefts[:len(forwards)]
rights = rights[:len(forwards)]

final_data = forwards + lefts + rights
shuffle(final_data)